In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existin

In [16]:
!pip install -q pycocotools

In [17]:
import os
import json
from pathlib import Path
from tqdm import tqdm
from pycocotools.coco import COCO

def convert_coco_pose_to_yolo(coco_json_path, image_dir, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    coco = COCO(coco_json_path)
    img_ids = coco.getImgIds(catIds=[1])  # only person

    for img_id in tqdm(img_ids, desc=f"Converting {coco_json_path}"):
        img_info = coco.loadImgs(img_id)[0]
        ann_ids = coco.getAnnIds(imgIds=img_id, catIds=[1], iscrowd=None)
        anns = coco.loadAnns(ann_ids)

        label_lines = []

        for ann in anns:
            if 'keypoints' not in ann or ann['num_keypoints'] == 0:
                continue

            x, y, w, h = ann['bbox']
            cx = x + w / 2
            cy = y + h / 2

            img_w = img_info['width']
            img_h = img_info['height']
            cx /= img_w
            cy /= img_h
            w /= img_w
            h /= img_h

            kpts = ann['keypoints']
            kpt_str = ''
            for i in range(17):
                px = kpts[i * 3] / img_w
                py = kpts[i * 3 + 1] / img_h
                v = kpts[i * 3 + 2]
                kpt_str += f' {px:.6f} {py:.6f} {v}'

            label_lines.append(f'0 {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}{kpt_str}')

        if label_lines:
            label_file = Path(img_info['file_name']).stem + '.txt'
            with open(os.path.join(save_dir, label_file), 'w') as f:
                f.write('\n'.join(label_lines))

# Đường dẫn
coco_dir = "/kaggle/input/coco-2017-keypoints/coco2017"
output_labels_dir = "/kaggle/working/datasets/labels"

# Convert train
convert_coco_pose_to_yolo(
    coco_json_path=os.path.join(coco_dir, "annotations/person_keypoints_train2017.json"),
    image_dir=os.path.join(coco_dir, "train2017"),
    save_dir=os.path.join(output_labels_dir, "train2017")
)

# Convert val
convert_coco_pose_to_yolo(
    coco_json_path=os.path.join(coco_dir, "annotations/person_keypoints_val2017.json"),
    image_dir=os.path.join(coco_dir, "val2017"),
    save_dir=os.path.join(output_labels_dir, "val2017")
)

loading annotations into memory...
Done (t=9.15s)
creating index...
index created!


Converting /kaggle/input/coco-2017-keypoints/coco2017/annotations/person_keypoints_train2017.json: 100%|██████████| 64115/64115 [00:07<00:00, 9151.03it/s]


loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


Converting /kaggle/input/coco-2017-keypoints/coco2017/annotations/person_keypoints_val2017.json: 100%|██████████| 2693/2693 [00:00<00:00, 10171.95it/s]


In [11]:
import shutil
import os

input_img_dir = "/kaggle/input/coco-2017-keypoints/coco2017/train2017"
output_img_dir = "/kaggle/working/datasets/images/train2017"
os.makedirs(output_img_dir, exist_ok=True)

for file in os.listdir(input_img_dir):
    if file.endswith(".jpg"):
        shutil.copy(os.path.join(input_img_dir, file), output_img_dir)

input_img_dir = "/kaggle/input/coco-2017-keypoints/coco2017/val2017"
output_img_dir = "/kaggle/working/datasets/images/val2017"
os.makedirs(output_img_dir, exist_ok=True)

for file in os.listdir(input_img_dir):
    if file.endswith(".jpg"):
        shutil.copy(os.path.join(input_img_dir, file), output_img_dir)

In [15]:
yaml_content = """
path: /kaggle/working/datasets

train: images/train2017
val: images/val2017

kpt_shape: [17, 3]
flip_idx: [0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 12, 11, 14, 13, 16, 15]

names:
  0: person
"""

with open('/kaggle/working/coco_pose.yaml', 'w') as f:
    f.write(yaml_content.strip())


In [18]:
# Xác nhận file đã tạo
!cat /kaggle/working/coco_pose.yaml

path: /kaggle/working/datasets

train: images/train2017
val: images/val2017

kpt_shape: [17, 3]
flip_idx: [0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 12, 11, 14, 13, 16, 15]

names:
  0: person

In [ ]:
from ultralytics import YOLO

# Tải mô hình pose YOLOv8 nano
model = YOLO('yolov8n-pose.pt')

model.train(
    data='/kaggle/working/coco_pose.yaml',
    epochs=150,         # hoặc 150 nếu bạn muốn dùng hết 30 giờ
    imgsz=640,
    batch=64,           # nếu đủ VRAM (~8GB trở lên với YOLOv8n)
    cache=True,         # nếu RAM >= 30GB
    device=0,
    workers=2,
    patience=20,         # tự dừng nếu không cải thiện sau 20 epochs
    save_period=1
)


Ultralytics 8.3.133 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/coco_pose.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pretrai

train: Scanning /kaggle/working/datasets/labels/train2017.cache... 56599 images, 0 backgrounds, 0 corrupt: 100%|██████████| 56599/56599 [00:00<?, ?it/s]


WARNING ⚠️ train: 69.3GB RAM required to cache images with 50% safety margin but only 25.1/31.4GB available, not caching images
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 813.0±536.3 MB/s, size: 128.6 KB)


val: Scanning /kaggle/working/datasets/labels/val2017.cache... 2346 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2346/2346 [00:00<?, ?it/s]


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


...


      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     33/150      8.08G      1.056      3.859     0.3473     0.9213      1.213        143        640: 100%|██████████| 885/885 [10:35<00:00,  1.39it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:17<00:00,  1.09it/s]

                   all       2346       6352       0.86      0.796      0.893      0.672      0.828      0.707      0.769      0.462



      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


     34/150      8.21G      1.058      3.849     0.3476     0.9192      1.213        349        640:  83%|████████▎ | 733/885 [08:44<01:58,  1.29it/s]

## Predict

In [ ]:
results = model.predict('/kaggle/input/input-poses/baseball1.jpg', save=True)  # predict on an image

## Display Result

In [ ]:
from IPython.display import Image

In [ ]:
Image(filename='runs/pose/predict/baseball1.jpg')